<a href="https://colab.research.google.com/github/DajeanArcila/DajeanArcila/blob/main/Churn_clasifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preveja os usuários com alta chance de deixar seu Streaming


In [454]:
#importamos as bibliotecas necesarias para manipulação, tratativa, clasificação e para medir as nosas metricas

#tratativa de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import requests
from io import StringIO


#medição de metricas
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

#modelo de ML|RN
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier


In [455]:
df= pd.read_csv('/content/streaming_data.csv')
df.head()

,Age,Gender,Time_on_platform,Devices_connected,Subscription_type,Num_streaming_services,Num_active_profiles,Avg_rating,Churned,User_id
0,49.0,Male,4700.0,3.0,Basic,NaN,3,NaN,0.0,a271aa56-bcfc-4f0a-91f7-b773de6b86a4
1,69.0,Female,6107.0,2.0,Standard,NaN,1,2.14,0.0,fe14b048-770b-4619-8ec6-485b0969ae31
2,44.0,Male,6374.0,3.0,Premium,1.0,1,1.91,1.0,0283dffd-6684-4a05-9c6f-c02098a6849d
3,26.0,Male,NaN,5.0,Basic,NaN,2,NaN,NaN,c316f2f5-3403-4a0a-82c2-c98e4b3379d2
4,18.0,NaN,1948.0,3.0,Basic,NaN,3,3.88,0.0,14879bcb-1977-4ad8-a7f9-6aa9bf7b872f


In [456]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77884 entries, 0 to 77883
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     66607 non-null  float64
 1   Gender                  63368 non-null  object 
 2   Time_on_platform        56464 non-null  float64
 3   Devices_connected       52696 non-null  float64
 4   Subscription_type       52663 non-null  object 
 5   Num_streaming_services  9572 non-null   float64
 6   Num_active_profiles     77884 non-null  int64  
 7   Avg_rating              60614 non-null  float64
 8   Churned                 61148 non-null  float64
 9   User_id                 77884 non-null  object 
dtypes: float64(6), int64(1), object(3)
memory usage: 5.9+ MB


In [457]:
df.isna().sum()

,0
Age,11277
Gender,14516
Time_on_platform,21420
Devices_connected,25188
Subscription_type,25221
Num_streaming_services,68312
Num_active_profiles,0
Avg_rating,17270
Churned,16736
User_id,0


In [458]:
#Posivelmente esta variavel nao afete muito a nossa analise, igual aqui apresento uma opcao para resolver este problema
'''
generos = ['Male', 'Female']

df['Gender'] = df['Gender'].apply(lambda x: np.random.choice(generos) if pd.isna(x) else x)
'''

"\ngeneros = ['Male', 'Female']\n\ndf['Gender'] = df['Gender'].apply(lambda x: np.random.choice(generos) if pd.isna(x) else x)\n"

In [459]:
df['Avg_rating'].fillna(0, inplace=True)

df['Num_streaming_services'].fillna(0, inplace=True)

df['Devices_connected'].fillna(0,inplace=True)

df['Time_on_platform'].fillna(0,inplace=True)

df['Churned'].fillna(0,inplace=True)



In [460]:
df.isna().sum()

,0
Age,11277
Gender,14516
Time_on_platform,0
Devices_connected,0
Subscription_type,25221
Num_streaming_services,0
Num_active_profiles,0
Avg_rating,0
Churned,0
User_id,0


In [461]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77884 entries, 0 to 77883
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     66607 non-null  float64
 1   Gender                  63368 non-null  object 
 2   Time_on_platform        77884 non-null  float64
 3   Devices_connected       77884 non-null  float64
 4   Subscription_type       52663 non-null  object 
 5   Num_streaming_services  77884 non-null  float64
 6   Num_active_profiles     77884 non-null  int64  
 7   Avg_rating              77884 non-null  float64
 8   Churned                 77884 non-null  float64
 9   User_id                 77884 non-null  object 
dtypes: float64(6), int64(1), object(3)
memory usage: 5.9+ MB


In [462]:
# Ver la distribución de valores en una columna después de la imputación
print(df['Gender'].value_counts())
print(df['Subscription_type'].value_counts())

Gender
Male      31796
Female    31572
Name: count, dtype: int64
Subscription_type
Premium     17625
Basic       17527
Standard    17511
Name: count, dtype: int64


In [463]:
df['Churned'].value_counts()

,count
Churned,
0.0,63021
1.0,14863


In [464]:
# Eliminar nulos por columna individualmente y observar el efecto en los datos
#df = df.dropna(subset=['Gender'])
#df = df.dropna(subset=['Subscription_type'])
#df = df.dropna(subset=['Age'])
df.dropna(inplace=True)

In [465]:
# Convertir todas las columnas float a int
df = df.astype({'Time_on_platform': 'int', 'Devices_connected': 'int','Num_streaming_services':'int','Avg_rating':'int','Age':'int','Churned':'int'})

In [466]:
df['Churned'] = df['Churned'].replace({0: 'No', 1: 'yes'})

In [467]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36515 entries, 0 to 77883
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     36515 non-null  int64 
 1   Gender                  36515 non-null  object
 2   Time_on_platform        36515 non-null  int64 
 3   Devices_connected       36515 non-null  int64 
 4   Subscription_type       36515 non-null  object
 5   Num_streaming_services  36515 non-null  int64 
 6   Num_active_profiles     36515 non-null  int64 
 7   Avg_rating              36515 non-null  int64 
 8   Churned                 36515 non-null  object
 9   User_id                 36515 non-null  object
dtypes: int64(6), object(4)
memory usage: 3.1+ MB


In [468]:
df['Churned'].value_counts()

,count
Churned,
No,29459
yes,7056


In [484]:
cat_vars = ["Gender","User_id","Churned","Subscription_type"]
num_vars = ["Age","Time_on_platform","Devices_connected","Num_streaming_services","Avg_rating"]
modeling_vars = cat_vars + num_vars

In [ ]:
LogisticRegression